In [4]:
import os
os.chdir("../..")
from sibr_module import BigQuery, Logger, CStorage
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings("ignore", category=UserWarning, message=".*feature names, but LGBMRegressor was fitted with feature names*")
import seaborn as sns
import matplotlib.pyplot as plt
from src.sibr_market_training import Train
from catboost import CatBoostRegressor

In [3]:
dataset = 'cars'
logger = Logger(f'train{dataset.capitalize()}')
bq = BigQuery(logger=logger,dataset=dataset)
cs = CStorage(logger=logger, bucket_name='sibr-market-bucket')
t = Train(dataset = dataset,logger = logger, log_target=True)
save_to_gc = True

2025-08-03 13:48:03,371 - trainCars - INFO - Cloud Logging is disabled. Using local logging to /Users/sigvardbratlie/Documents/Projects/sibr_market_training/logfiles/trainCars.log
2025-08-03 13:48:03,375 - trainCars - INFO - BigQuery client initialized with project_id: sibr-market
2025-08-03 13:48:03,377 - trainCars - INFO - Google Cloud Storage client initialized with bucket: sibr-market-bucket
2025-08-03 13:48:03,380 - trainCars - INFO - BigQuery client initialized with project_id: sibr-market
2025-08-03 13:48:03,382 - trainCars - INFO - Google Cloud Storage client initialized with bucket: sibr-market
2025-08-03 13:48:03,382 - trainCars - DEBUG - Dataset: cars | | Replace: False


In [ ]:
data = t.bq.read_cars(task = "train")
df_el = data.get("cars_el")
df_fossil = data.get("cars_fossil")

2025-08-03 13:49:19,543 - trainCars - INFO - 135218 rows read from cars. Query: SELECT a.* FROM `sibr-market.pre_processed.cars_el` a... (truncated)
2025-08-03 13:49:23,088 - trainCars - INFO - 355354 rows read from cars. Query: SELECT a.* FROM `sibr-market.pre_processed.cars_fossil` a... (truncated)


In [7]:
df_el.dropna(inplace=True)
df_fossil.dropna(inplace=True)
t.logger.info(f'Length dataframes after dropping NaN:  df_el: {len(df_el)} | df_fossil: {len(df_fossil)}')

2025-08-03 13:54:10,210 - trainCars - INFO - Length dataframes after dropping NaN:  df_el: 135197 | df_fossil: 346289


In [8]:
params_el = {'depth': 7, 'iterations': 1223, 'l2_leaf_reg': np.float64(1.3895264494261033),
                'learning_rate': np.float64(0.09150423569345205), 'random_state': 52}
params_fossil = {'depth': 4, 'iterations': 1463, 'l2_leaf_reg': np.float64(1.7419090119209117),
                    'learning_rate': np.float64(0.10215580871496355), 'random_state': 43}

pipeline_el = t.train(df = df_el,
                                params = params_el,
                                target = 'total_price',
                                data_name =  'cars_el',
                                model = CatBoostRegressor,
                                save_to_gc=save_to_gc,
                                categorical=True,
                                log_target = True)
pipeline_fossil = t.train(df = df_fossil,
                                    params = params_fossil,
                                    target = 'total_price',
                                    data_name = 'cars_fossil',
                                    model = CatBoostRegressor,
                                    save_to_gc = save_to_gc
                                    ,categorical=True,
                                    log_target = True)

2025-08-03 13:54:15,496 - trainCars - INFO - 
 
TRAINING CatBoostRegressor model for CARS_EL
2025-08-03 13:54:15,571 - trainCars - INFO - Train set size: 108157, Test set size: 27040
2025-08-03 13:54:15,572 - trainCars - INFO - Target: total_price and log_target: True
2025-08-03 13:54:45,144 - trainCars - INFO - MSE test: 760354464.2396675,r2 test: 0.9798060812320136, mse train: 562008656.1450939, r2 train 0.9849059324876347 for cars_el with target total_price and log_target True
2025-08-03 13:54:45,523 - trainCars - INFO - Blob models.json downloaded to /tmp/models.json.
2025-08-03 13:54:45,815 - trainCars - INFO - File /tmp/models.json uploaded to models.json in bucket sibr-market.
2025-08-03 13:54:46,304 - trainCars - INFO - File /tmp/tmp_file.pkl uploaded to models/CatBoostRegressor_cars_el.pkl in bucket sibr-market.
2025-08-03 13:54:46,320 - trainCars - INFO - 
 
TRAINING CatBoostRegressor model for CARS_FOSSIL
2025-08-03 13:54:46,464 - trainCars - INFO - Train set size: 277031, T